In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip  --quiet
! pip install mlflow --quiet
! pip install pyngrok --quiet
! pip install pulp --quiet

import IPython
import pandas as pd
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import mlflow
from pyngrok import ngrok


#01.Mount drive

In [ ]:
from google.colab import drive

mountPoint='/content/gdrive'
drive.mount(mountPoint)
DataFolder=f'{mountPoint}/MyDrive/ColabNotebooks/FPL/Data'

# 02.Import Data

In [ ]:
%run /content/gdrive/My\ Drive/ColabNotebooks/FPL/01_ImportData.ipynb

In [ ]:
import os.path
if os.path.isfile("{0}/GWSummary.parquet".format(DataFolder))==False:
  print("> Importing Game Weeks...")
  ImportGameWeeks(3, DataFolder) # Importing last 3 game weeks data
if os.path.isfile("{0}/PlayersRaw.parquet".format(DataFolder))==False:
  print("> Importing Players raw data...")
  ImportPlayersRaw(DataFolder) # Import Players Raw Data

# 03.Profiling Data

In [ ]:
%run /content/gdrive/My\ Drive/ColabNotebooks/FPL/02_ProfileDataSet.ipynb

In [ ]:

if os.path.isfile("{0}/DataProfile.html".format(DataFolder))==False:
  df_profile=ProfileGW(DataFolder)
  df_profile

In [ ]:
experiment_id = mlflow.create_experiment("FPL Team Recommend")

# 04. Linear Programming

In [ ]:
%run /content/gdrive/My\ Drive/ColabNotebooks/FPL/03_LinearProgramming.ipynb

In [ ]:
df=pd.read_parquet(f"{DataFolder}/GWSummary.parquet")
df=df[df["total_points"]>0]
meanDEF=int(df[df["position"]=="DEF"]["total_points"].mean())
meanGK=int(df[df["position"]=="GK"]["total_points"].mean())
meanMID=int(df[df["position"]=="MID"]["total_points"].mean())
meanFWD=int(df[df["position"]=="FWD"]["total_points"].mean())

df2=pd.concat([ 
                  df[(df["position"]=="DEF") & (df["total_points"]>meanDEF)],
                  df[(df["position"]=="MID") & (df["total_points"]>meanMID)],
                  df[(df["position"]=="GK") & (df["total_points"]>meanGK)],
                  df[(df["position"]=="FWD") & (df["total_points"]>meanFWD)]
               
               ])

print(">> Original Length {0}, Filtered Length {1}".format(len(df), len(df2)))

In [ ]:
df=df2

MaxBudget=1000

for BudgetLimit in range(MaxBudget, int(0.7*MaxBudget), -50):

  SquadComposition={
    "Goalkeepers": 2,
    "Defenders":5,
    "Midfielders":5,
    "Forwards":3,
    "Sameteam":3,
    "MaxCost": BudgetLimit
  }

  with mlflow.start_run(experiment_id=experiment_id, run_name="LP_{0}".format(BudgetLimit ) ) as run:
    dfResult=SolveLP(df, SquadComposition, 3, BudgetLimit)
    x=(dfResult[["total_points", "value"]].sum())
    mlflow.log_param("Config", BudgetLimit)
    mlflow.log_metric("Score", x.iloc[0])
    mlflow.log_metric("Value", x.iloc[1])
    dfResult.sort_values(by=['position', 'total_points'], ascending=[True,False]).to_csv('{0}/mlflowLogs/score_LP_Budget_{1}.csv'.format(DataFolder,BudgetLimit), header=True, index=False)
    mlflow.log_artifact('{0}/mlflowLogs/score_LP_Budget_{1}.csv'.format(DataFolder, BudgetLimit ), "Selected Team")

# 05.Run Genetic Algorithm

In [ ]:
%run /content/gdrive/My\ Drive/ColabNotebooks/FPL/03_GeneticAlgorithm.ipynb

In [ ]:
import uuid
from multiprocessing import Pool

SquadComposition={
    "Goalkeepers": 2,
    "Defenders":5,
    "Midfielders":5,
    "Forwards":3,
    "Sameteam":3,
    "MaxCost": 1000
}

def funcProc(runTMP):
    runid=str(uuid.uuid1())
    ParamStr="Pop:{0},NGen:{1}".format(runTMP["PopSize"],runTMP["n_gen"] )
    print(f">> Starting experiment {runid} - {ParamStr}... ")
    Config={
      "PopSize":runTMP["PopSize"],
      "Nparents":runTMP["NParents"],
      "mutation_rate":runTMP["mutation_rate"],
      "crossover_ratio":runTMP["crossover_ratio"],
      "stopCriteria":16,
      "n_gen":runTMP["n_gen"],
      "MaxSquadsToPick":4,
      "ErrorAttempts":15,
      "SquadComposition":SquadComposition,
      "PenaltyPoints":{
        "elementNews":-10
      }
    }

    with mlflow.start_run(experiment_id=experiment_id, run_name="GA_{0}".format(runid ) ) as run:
      SquadSize=SquadComposition["Goalkeepers"]+SquadComposition["Defenders"]+SquadComposition["Midfielders"]+SquadComposition["Forwards"]
      df=df2
      
      gaAlg= GAlgorithm(Config, df)
      best_chromo, best_score, scoresTotal= gaAlg.Execute(runid)

      gaAlg.plotScores(DataFolder, runid)
      gaAlg.PlotSquads(DataFolder, runid)
      gaAlg.TopPlayers(DataFolder, runid)

      mlflow.log_param("Config", Config)
      mlflow.log_metric("Score", best_score[-1])
      mlflow.log_artifact('{0}/mlflowLogs/GA_scores_{1}.png'.format(DataFolder, runid ), "Score per Generation")
      mlflow.log_artifact('{0}/mlflowLogs/GA_TopPlayers_{1}.csv'.format(DataFolder, runid), "Best Players")
      mlflow.log_artifact('{0}/mlflowLogs/GA_Teams_{1}.csv'.format(DataFolder, runid ), "Best Teams")
    
    
    return



runList=[
         {"PopSize": 200, "NParents":70, "n_gen":40, "mutation_rate": 0.2, "crossover_ratio": 0.3},
         {"PopSize": 50, "NParents":10, "n_gen":20, "mutation_rate": 0.2, "crossover_ratio": 0.3},
         {"PopSize": 50, "NParents":10, "n_gen":20, "mutation_rate": 0.3, "crossover_ratio": 0.2},
         {"PopSize": 200, "NParents":50, "n_gen":20, "mutation_rate": 0.3, "crossover_ratio": 0.1},
         {"PopSize": 100, "NParents":30, "n_gen":80, "mutation_rate": 0.4, "crossover_ratio": 0.4}
]

with Pool(5) as p:
  z=p.map(funcProc, runList)






# 06. Tunnel to MLFLow web app

In [ ]:
# Terminate open tunnels if exist
ngrok.kill()

# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = 'xxxxxx' #@param {type:"string"}
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
get_ipython().system_raw("mlflow ui --port 5000 &")
# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)